In [8]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from karateclub.dataset import GraphSetReader
from karateclub import FeatherGraph
from sklearn.model_selection import train_test_split
import networkx as nx

import itertools

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

import praw
from praw.models import MoreComments

from utils.utils import *

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
client_id = '8Ara-tL3whPJTKSzuLe0Wg'
client_secret = '5ciIZ_875ywNSWHWY7S4QcaimF788g'
user_agent = '<console:IC_proiect:1.0>'

reddit = praw.Reddit(client_id = client_id,
                    client_secret = client_secret,
                    user_agent = user_agent)

In [4]:
subredditFood = reddit.subreddit("food")
subredditHardware = reddit.subreddit("hardware")

In [5]:
def comments_to_ids(comments, name_to_id):
    l = []
    for comment in comments:
        if comment.author == None:
            continue
        id = name_to_id.get(comment.author)
        if id == None:
            lg = len(name_to_id)
            name_to_id[comment.author] = lg;
            id = lg
        l.append(id)
    return l


def get_graph_indices_from_submission(G, submission, limit=None):
    submission.comments.replace_more(limit=limit)

    graph_list = []
    comment_queue = submission.comments[:]

    name_to_id = {}
    name_to_id[submission.author] = 0
    
    nodes_list = [{}]

    l = comments_to_ids(comment_queue, name_to_id)
    graph_list = list(zip(itertools.repeat(0), l))
    
    G.add_edges_from(list(zip(itertools.repeat(0), l)))
    G.add_node(0)

    while comment_queue:
        comment = comment_queue.pop(0)

        if comment.author == None:
            continue

        comment_replies = list(comment.replies)
        comment_queue.extend(comment_replies)

        l = comments_to_ids(comment_replies, name_to_id)
        this_id = comments_to_ids([comment], name_to_id)[0]
        graph_list += list(zip(itertools.repeat(this_id), l))
        
        G.add_edges_from(list(zip(itertools.repeat(this_id), l)))
        G.add_node(this_id)
        G.nodes[this_id]["score"] = comment.score

    return list(range(len(graph_list))), graph_list

post_url = "https://www.reddit.com/r/food/comments/urzgdv/homemade_sushi/"
submission = reddit.submission(url = post_url)

G = nx.Graph()

get_graph_indices_from_submission(G, submission)

# G.add_edges_from(edges)
# G.add_nodes_from(nodes)

plt.figure(figsize = (10, 10))
#nx.draw_kamada_kawai(G, with_labels=True, width = .5)

G.nodes.data, G.edges.data

(<bound method NodeView.data of NodeView((0, 1, 2, 3, 4, 5, 6, 7))>,
 <bound method OutEdgeView.data of EdgeView([(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 7), (5, 6)])>)

<Figure size 720x720 with 0 Axes>

In [6]:
def get_graphs_from_subreddit(subreddit, limit=10):
    list = []
    for submission in subreddit.hot(limit=limit):
        print(submission.title)

        G = nx.Graph()
        get_graph_indices_from_submission(G, submission, limit=100)

        if(G.edges == []):
            continue
        
        # G.add_edges_from(edges)
        # G.add_nodes_from(nodes)

        print(G.number_of_nodes())

        list.append(G)
    return list

In [7]:
graphsFood = get_graphs_from_subreddit(subredditFood, 50)

[Homemade] bolognese
5
[i ate] ice cream from an ice cream truck.
17
[homemade] Smash Burger.
12
[homemade] Peanut Butter & Jelly Cookies
109
[I Ate] A Loaded Chef Salad.
45
[i ate] knuckle of pork with sauerkraut and dumplings
180
[homemade] Garlic Parmesan Encrusted Pearl Onions
86
[homemade] spicy pasta
19
[Homemade] Smoked Butterflied Drumsticks
5
[Homemade] Bacon Cheeseburger
4
[Pro/Chef] Mushroom prep. Nature is beautiful.
10
[homemade] Cheese Pizza
7
[homemade] Pepperoni Pizza
2
[Homemade] Lunch pizza
7
[homemade] leftover meatloaf air fried and air fried mashed potato patty sandwich with a Kraft single and some ketchup
39
[Homemade] Brisket and Egg Sando
62
[I ate] Texas BBQ Platter
29
[homemade] Falafel balls with tzatziki sauce
2
[Homemade] Chocolate and orange muffin
1
[homemade] pork, chicken, sausage, mushroom, spinach paella
3
[homemade] Aglio Olio with shrimp
3
[Homemade] Hawaiian hotdogs with grilled pineapple and a lime Teriyaki sauce with a side of Hawaiian mac salad


In [8]:
graphsHardware = get_graphs_from_subreddit(subredditHardware, 50)

Reminder: Please do not submit tech support or build questions to /r/hardware
9
About Videocardz and Original Sources
36
Framework Laptop upgrades to 12th gen Intel
15
[PR] GeIL launches world’s first DDR5 memory with dual RGB fans, EVO-V available up to DDR5-6600 64GB kits starting July
36
Radeon Preview Driver, 50 Game Benchmark [HUB]
22
NVIDIA GeForce GTX 1630 to feature 512 CUDA cores, 4GB 64-bit memory, launches May 31st - VideoCardz.com
18
Tested: AMD’s Ryzen 7 6800U brings big performance to the tiniest laptops | PCWorld
3
Acer gaming laptop offers glasses-free 3D powered by an Nvidia RTX 3080 GPU
66
Let's Examine AMD's Claim That Radeons Deliver More FPS Per Dollar Than GeForce
92
[AnandTech] The SilentiumPC Fera 5 CPU Cooler Review: Quiet Cooling on a Budget
11
Berkeley Lab: "New Silicon Nanowires Can Really Take the Heat"
6
AMD's FSR 2.0 Even Worked With Intel Integrated Graphics
27
EzBench, a free Unreal Engine 5 gaming benchmark, is now available for free on Steam. An easy 

In [9]:
model = FeatherGraph()

graphs = graphsFood + graphsHardware

# plt.figure(figsize = (10, 10))
# nx.draw_kamada_kawai(graphs[11], with_labels=True, width = .5)

model.fit(graphs)
X = model.get_embedding()

In [10]:
y = [0] * len(graphsFood) + [1] * len(graphsHardware)

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [104]:
downstream_model = LogisticRegression(max_iter=500, random_state=0).fit(X_train, y_train)
#y_hat = downstream_model.predict_proba(X_test)[:, 1]

print(downstream_model.score(X_test, y_test))

0.65


In [20]:
#post_url = "https://www.reddit.com/r/food/comments/uspd0m/i_ate_texas_bbq_platter/"
post_url = "https://www.reddit.com/r/food/comments/ushi7j/homemade_garlic_parmesan_encrusted_pearl_onions/"
submission = reddit.submission(url = post_url)

G = nx.Graph()

nodes, edges = get_graph_indices_from_submission(G, submission)

# G.add_edges_from(edges)
# G.add_nodes_from(nodes)

model.fit([G])
X_eval = model.get_embedding()

y_pred = downstream_model.predict_proba(X_eval)
y_pred

array([[0.92522965, 0.07477035]])